Este cuaderno es una demo del modulo _Simulador_ y el modulo de optizacion (_Workforce_), que usa este para computar una estrategia optima. 

# Simulador (noviembre 2023)
## para simulación: Clase ``MisEscritorios_v03`` y funcion ``simulador_v04``
## In progress: para optuna: Clase ``MisEscritorios_v02`` y funcion ``optuna_simular_v02``

- `MisEscritorios_v03` contiene todo lo respectivo a planificación. 
- `simulador_v04` es la función que corre una simulación, generando una tabla de atenciones. 


## Simulación con Clase ``MisEscritorios_v03`` y funcion ``simulador_v04``

In [3]:
import numpy as np
from datetime import datetime
from dev.atributos_de_series import atributos_x_serie
from src.datos_utils import DatasetTTP, obtener_skills
import random

from src.optuna_utils import (
    sla_x_serie, 
    calculate_geometric_mean, 
    extract_skills_length, 
    extract_min_value_keys, 
    extract_max_value_keys, 
    non_empty_subsets, 
    get_random_non_empty_subset, 
    get_time_intervals,
    partition_dataframe_by_time_intervals,  
    plan_unico
    )

from src.simulador_v02 import (
    one_cycle_iterator,
    create_multiindex_df,
    generate_integer, 
    actualizar_conexiones,
    generador_emisiones,
    timestamp_iterator,
    terminar_un_tramo,
    iniciar_un_tramo,
    update_escritorio,
    separar_por_conexion,
    poner_pasos_alternancia,
    pasos_alternancia,
    mismo_minuto,
    balancear_carga_escritorios,
    extract_highest_priority_and_earliest_time_row,
    remove_selected_row,
    FIFO
    ) 
import pandas as pd

from datetime import timedelta
from dev.atributos_de_series import atributos_x_serie
from dev.pasos_alternancia_y_prioridades_x_escri import (
    generar_pasos_para_alternancia_v02, 
    pasos_alternancia_v02,
    poner_pasos_alternancia_v02,
    MisEscritorios_v03,        
)

from dev.simv04 import (
    reloj_rango_horario, 
    match_emisiones_reloj, 
    simulador_v04)

dataset = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
skills   = obtener_skills(un_dia)
series   = sorted(list({val for sublist in skills.values() for val in sublist}))
modos    = ['FIFO']#['Rebalse','Alternancia', 'Rebalse']
atributos_series = atributos_x_serie(ids_series=series, 
                                    sla_porcen_user=None, 
                                    sla_corte_user=None, 
                                    pasos_user=None, 
                                    prioridades_user=None)

niveles_servicio_x_serie = {atr_dict['serie']:
                            (atr_dict['sla_porcen']/100, atr_dict['sla_corte']/60) 
                            for atr_dict in atributos_series}

prioridades =       {atr_dict['serie']:
                        atr_dict['prioridad']
                        for atr_dict in atributos_series}   


planificacion = {'0': [{'inicio': '08:00:11',
    'termino': None,
    'propiedades': {'skills' : get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,
            
        }}],
    '1': [{'inicio': '08:00:11',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '12': [{'inicio': '08:00:11',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '33': [{'inicio': '08:00:03',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '34': [{'inicio': '08:00:03',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '35': [{'inicio': '08:00:03',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '49': [{'inicio': '08:00:56',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series), 
        'configuracion_atencion':random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '50': [{'inicio': '08:00:56',
    'termino': None,
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '51': [{'inicio': '08:00:56',
    'termino': None,
    'propiedades': {'skills':get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(85, 90)/100,
        'atributos_series':atributos_series,
        }}]}

hora_cierre               = '20:00:00'
registros_atenciones, fila = simulador_v04(un_dia, planificacion, niveles_servicio_x_serie, hora_cierre)

len(registros_atenciones) , len(fila)

(651, 8)

## IN PROGRESS : OPTUNA con Clase ``MisEscritorios_v02`` y funcion ``optuna_simular_v02``

# Objetivos optuna (broken)

In [29]:

import optuna
import numpy as np
dataset = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
skills   = obtener_skills(un_dia)
series   = sorted(list({val for sublist in skills.values() for val in sublist}))
modos    = ['Rebalse','Alternancia', 'Rebalse']
atributos_series = atributos_x_serie(ids_series=series, 
                                    sla_porcen_user=None, 
                                    sla_corte_user=None, 
                                    pasos_user=None, 
                                    prioridades_user=None)

niveles_servicio_x_serie = {atr_dict['serie']:
                           (atr_dict['sla_porcen']/100, atr_dict['sla_corte']/60) 
                           for atr_dict in atributos_series}

prioridades =       {atr_dict['serie']:
                    atr_dict['prioridad']
                    for atr_dict in atributos_series}

def objective(trial, 
    optimizar: str, 
    un_dia : pd.DataFrame,  # IdOficina  IdSerie  IdEsc, FH_Emi, FH_Llama  -- Deberia llamarse 'un_tramo'
    subsets, # [(5,), (10,), (11,), (12,), (14,), (17,), (5, 10), (5, 11), (5, 12), (5, 14), (5, 17), (10, 11),  <...> 14, 17), (5, 10, 12, 14, 17), (5, 11, 12, 14, 17), (10, 11, 12, 14, 17), (5, 10, 11, 12, 14, 17)]
    niveles_servicio_x_serie,  # {5: (0.34, 35), 10: (0.34, 35), 11: (0.7, 45), 12: (0.34, 35), 14: (0.34, 35), 17: (0.6, 30)}
    prioridades:dict,
    modos_atenciones : list = ["Alternancia", "FIFO", "Rebalse"],
    minimo_escritorios: int = 2,
    maximo_escritorios: int = 5,
    ):    
    try:

        bool_vector              = [trial.suggest_categorical(f'escritorio_{i}', [True, False]) for i in range(maximo_escritorios)]
        #Restricción de minimo de escritorios
        assert sum(bool_vector) >= minimo_escritorios, f"No cumple con minimo_escritorios: {minimo_escritorios}."
        
        str_dict                 = {i: trial.suggest_categorical(f'{i}',         modos_atenciones) for i in range(maximo_escritorios)} 
        subset_idx               = {i: trial.suggest_int(f'ids_{i}', 0, len(subsets) - 1) for i in range(maximo_escritorios)}   
        #prioridades              =  prioridad_x_serie(niveles_servicio_x_serie, 2, 1) 
        planificacion            =  {} # Arma una planificacion con espacios parametricos. 
        inicio                   =  str(un_dia.FH_Emi.min().time())#'08:33:00'
        termino                  =  str(un_dia.FH_Emi.max().time())#'14:33:00'
        #porcentaje_actividad     =  np.random.randint(75, 90)/100
        for key in str_dict.keys():
            if bool_vector[key]:
                inner_dict = {
                    'inicio': inicio,
                    'termino': termino,
                    'propiedades': {
                        'skills':list(subsets[subset_idx[key]]), # Set -> Lista, para el subset 'subset_idx', para el escritorio 'key'
                        'configuracion_atencion': str_dict[key], # 
                    }
                }
                planificacion[str(key)] = [inner_dict] # NOTE: Es una lista why -- Config por trial por tramo del escritorio 

        trial.set_user_attr('planificacion', planificacion) # This' actually cool 
        registros_atenciones, l_fila    =  optuna_simular_v02(planificacion, niveles_servicio_x_serie, un_dia, prioridades, tipo_inactividad = "historica") 
        registros_atenciones['IdSerie'] = registros_atenciones['IdSerie'].astype(int) 
        registros_x_serie               = [registros_atenciones[registros_atenciones.IdSerie==s] for s in series]
        
        
        pocentajes_SLA        = [int(100*v[0])for k,v in niveles_servicio_x_serie.items()]
        mins_de_corte_SLA     = [int(v[1])for k,v in niveles_servicio_x_serie.items()]        
        df_pairs              = [(sla_x_serie(r_x_s, '1H', corte = corte, factor_conversion_T_esp=1), s) 
                                    for r_x_s, s, corte in zip(registros_x_serie, series, mins_de_corte_SLA)]
        porcentajes_reales    = {f"serie: {serie}": np.mean(esperas.espera) for ((demandas, esperas), serie) in df_pairs} 
        dif_cuadratica        = {k:(v-p)**2 for ((k,v),p) in zip(porcentajes_reales.items(),pocentajes_SLA)}
        #Objetivos:    
        #La mayor prioridad es el entero más chico    
        maximizar_SLAs        = tuple(np.array(tuple(prioridades.values()))*np.array(tuple(dif_cuadratica.values())))#Ponderado por prioridad
        minimizar_escritorios = (sum(bool_vector),)
        minimizar_skills      = (extract_skills_length(planificacion),)
        
        if optimizar == "SLA":
            
            print(f"maximizar_SLAs {maximizar_SLAs}")
            return  maximizar_SLAs
        
        elif optimizar == "SLA + escritorios":
            
            print(f"maximizar_SLAs y minimizar_escritorios {maximizar_SLAs, minimizar_escritorios}")
            return  maximizar_SLAs + minimizar_escritorios
        
        elif optimizar == "SLA + skills":
            
            print(f"maximizar_SLAs y minimizar_skills {maximizar_SLAs, minimizar_skills}")
            return  maximizar_SLAs + minimizar_skills
        
        elif optimizar == "SLA + escritorios + skills":
            
            print(f"SLA + escritorios + skills {maximizar_SLAs, minimizar_escritorios, minimizar_skills}")
            return  maximizar_SLAs + minimizar_escritorios + minimizar_skills           
        
    except Exception as e:
        print(f"An exception occurred: {e}")
        raise optuna.TrialPruned()
    
#Si hay porcentaje_actividad no hay pausas  
#n es el numero de intervalos (equidistantes) de tiempo   
intervals  = get_time_intervals(un_dia, n = 4, porcentaje_actividad = .9) # Una funcion que recibe un dia, un intervalo, y un porcentaje de actividad para todos los intervalos
partitions = partition_dataframe_by_time_intervals(un_dia, intervals) # TODO: implementar como un static del simulador? 
optimizar  = "SLA + escritorios + skills" #"SLA" | "SLA + escritorios" | "SLA + skills" | "SLA + escritorios + skills"
n_objs = int(
            len(series)
            if optimizar == "SLA"
            else len(series) + 1
            if optimizar in {"SLA + escritorios", "SLA + skills"}
            else len(series) + 2
            if optimizar == "SLA + escritorios + skills"
            else None
        )
n_trials   = 1
#%%
storage = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db")
subsets = non_empty_subsets(sorted(list({val for sublist in skills.values() for val in sublist})))

for idx, part in enumerate(partitions):
    study_name = f"tramo_{idx}"
    study = optuna.multi_objective.create_study(directions= n_objs*['minimize'],
                                                study_name=study_name,
                                                storage=storage, load_if_exists=True)
    # TODO: sacar fuera
    # Optimize with a timeout (in seconds)
    study.optimize(lambda trial: objective(trial,
                                           optimizar                = optimizar,
                                           un_dia                   = part,
                                           subsets                  = subsets,
                                           niveles_servicio_x_serie = niveles_servicio_x_serie,
                                           prioridades              = prioridades,
                                           minimo_escritorios       = 2,
                                           maximo_escritorios       = 5
                                           ),
                   n_trials  = n_trials, #int(1e4),  # Make sure this is an integer
                   timeout   = 2*3600,   #  hours
                   )  # 


[I 2023-10-30 19:29:12,275] Using an existing study with name 'tramo_0' instead of creating a new one.
[I 2023-10-30 19:29:13,751] Trial 34 finished with values: (10084.132231404958,) with parameters: {'escritorio_0': True, 'escritorio_1': True, 'escritorio_2': False, 'escritorio_3': False, 'escritorio_4': True, '0': 'Alternancia', '1': 'FIFO', '2': 'Rebalse', '3': 'FIFO', '4': 'Rebalse', 'ids_0': 13, 'ids_1': 57, 'ids_2': 19, 'ids_3': 13, 'ids_4': 4}.
[I 2023-10-30 19:29:13,830] Using an existing study with name 'tramo_1' instead of creating a new one.


SLA + escritorios + skills ((10084.132231404958, 90.19575253924302, 68.48148148148151, 13.444444444444583, 5.621107266435958, 312.11111111111126), (3,), (8,))


[I 2023-10-30 19:29:13,959] Trial 34 pruned. 
[I 2023-10-30 19:29:14,035] Using an existing study with name 'tramo_2' instead of creating a new one.


An exception occurred: No cumple con minimo_escritorios: 2.


[I 2023-10-30 19:29:16,398] Trial 34 finished with values: (5902.066326530613,) with parameters: {'escritorio_0': True, 'escritorio_1': True, 'escritorio_2': True, 'escritorio_3': True, 'escritorio_4': False, '0': 'Alternancia', '1': 'Rebalse', '2': 'Rebalse', '3': 'Alternancia', '4': 'FIFO', 'ids_0': 0, 'ids_1': 51, 'ids_2': 16, 'ids_3': 17, 'ids_4': 35}.
[I 2023-10-30 19:29:16,478] Using an existing study with name 'tramo_3' instead of creating a new one.


SLA + escritorios + skills ((5902.066326530613, 2692.01652892562, 867.0, 12769.0, 928.8674648183153, 513.7777777777777), (4,), (9,))


[I 2023-10-30 19:29:17,662] Trial 34 finished with values: (980.0,) with parameters: {'escritorio_0': False, 'escritorio_1': True, 'escritorio_2': True, 'escritorio_3': False, 'escritorio_4': True, '0': 'FIFO', '1': 'Alternancia', '2': 'FIFO', '3': 'Rebalse', '4': 'FIFO', 'ids_0': 52, 'ids_1': 52, 'ids_2': 52, 'ids_3': 47, 'ids_4': 0}.


SLA + escritorios + skills ((980.0, 350.810529537802, 13467.0, 7744.0, 10658.0, 2601.0), (3,), (9,))


In [30]:
study.get_trials()[0].user_attrs.get('planificacion')

{'2': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [10, 14], 'configuracion_atencion': 'FIFO'}}],
 '3': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 10, 14, 17],
    'configuracion_atencion': 'Alternancia'}}],
 '4': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 11, 17], 'configuracion_atencion': 'FIFO'}}],
 '7': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [12, 17],
    'configuracion_atencion': 'Alternancia'}}],
 '11': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 11, 12, 17],
    'configuracion_atencion': 'Rebalse'}}],
 '12': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [10, 11, 12],
    'configuracion_atencion': 'Rebalse'}}]}

## Extracción de la planificación óptima

In [31]:
recomendaciones_db   = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db") # Objetivos de 6-salidas
resumenes            = optuna.study.get_all_study_summaries(recomendaciones_db)
nombres              = [s.study_name for s in resumenes if "tramo_" in s.study_name]

scores_studios = {}
for un_nombre in nombres:
    un_estudio            = optuna.multi_objective.load_study(study_name=un_nombre, storage=recomendaciones_db)
    trials_de_un_estudio  = un_estudio.get_trials(deepcopy=False) #or pareto trials??
    scores_studios        = scores_studios | {f"{un_nombre}":
        { trial.number: np.mean([x for x in trial.values if x is not None]) 
                for
                    trial in trials_de_un_estudio if trial.state == optuna.trial.TrialState.COMPLETE}
                    } 
trials_optimos          = extract_min_value_keys(scores_studios) # Para cada tramo, extrae el maximo, 
planificaciones_optimas = {}   
for k,v in trials_optimos.items():
    un_estudio               = optuna.multi_objective.load_study(study_name=k, storage=recomendaciones_db)
    trials_de_un_estudio     = un_estudio.get_trials(deepcopy=False)
    planificaciones_optimas  = planificaciones_optimas | {f"{k}":
        trial.user_attrs.get('planificacion')#calcular_optimo(trial.values)
                for
                    trial in trials_de_un_estudio if trial.number == v[0]
                    }   

planificacion_optima                =  plan_unico([plan for tramo,plan in planificaciones_optimas.items()])


### simulación con planificación óptima

In [83]:

def ghost_client(un_dia, slack_seconds=7200):
    
    un_dia.sort_values(by='FH_Emi', inplace=True)
    un_dia.reset_index(drop=True, inplace=True)
    last_row = un_dia.iloc[-1].copy()
    # Modify datetime64[s] columns by adding 2 hours (7200 seconds)
    datetime_columns = ['FH_Emi', 'FH_Llama', 'FH_AteIni', 'FH_AteFin']
    for col in datetime_columns:
        last_row[col] += pd.Timedelta(seconds=slack_seconds)  # 2 hours = 7200 seconds

    return pd.concat([un_dia, pd.DataFrame(last_row).T], axis=0, ignore_index=True)



In [81]:
registros_atenciones_optima, l_fila_optima =  optuna_simular_v02(planificacion_optima, niveles_servicio_x_serie, un_dia_con_ghost_client, prioridades, tipo_inactividad = "historica") 

In [82]:
registros_atenciones_optima, l_fila_optima

(                  FH_Emi IdSerie espera IdEsc T_Ate
 0    2023-05-15 08:40:11      12      9     7    96
 1    2023-05-15 08:40:54      14      9    10   157
 2    2023-05-15 08:41:07      14      9    12   458
 3    2023-05-15 08:41:18      14      8    11   861
 4    2023-05-15 08:41:27      14      8    10   381
 ..                   ...     ...    ...   ...   ...
 655  2023-05-15 14:13:23      10    137   NaN   NaN
 656  2023-05-15 14:14:26      12    136   NaN   NaN
 657  2023-05-15 14:18:32      10    132   NaN   NaN
 658  2023-05-15 14:30:23      17    120   NaN   NaN
 659  2023-05-15 16:30:23      17      0   NaN   NaN
 
 [660 rows x 5 columns],
 343)

In [36]:
registros_atenciones_optima["IdEsc"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 659 entries, 0 to 658
Series name: IdEsc
Non-Null Count  Dtype 
--------------  ----- 
331 non-null    object
dtypes: object(1)
memory usage: 5.3+ KB
